In [77]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import speech_recognition as sr
import pyttsx3
import openai
import requests
import json
from bs4 import BeautifulSoup as bs
from difflib import SequenceMatcher



In [78]:



openai.api_key_path = "key.txt"
cur_model = "text-davinci-002"
img = ["image", "picture", "photo", "landscape", "painting", "drawing", "art"]


# Define Brave path
brave_path = "C:/Program Files/BraveSoftware/Brave-Browser-Nightly/Application/brave.exe"
options = webdriver.ChromeOptions()
options.binary_location = brave_path

# set dl options
prefs = {"download.default_directory": "C:/Users/thewa/Desktop/projects/computational_neuroscience/AI_ML/projects/mun_ching/crawlers/content"}

options.add_experimental_option("prefs", prefs)

# Create new automated instance of Brave
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.minimize_window()


C:\Users\thewa\AppData\Local\Temp\ipykernel_2212\1448262854.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [79]:
def callGPT(prompt):

        completion = openai.Completion.create(
                    model=cur_model,
                    prompt=prompt,
                    temperature=0,
                    frequency_penalty=0.5,
                    max_tokens=20,


                )
        
        return completion.choices[0].text
    

In [80]:

def getGame(response):
    
    s = soundex.getInstance()
    URL = "https://steamcommunity.com/id/yusufwadi/games?tab=all&xml=1"
    data = requests.get(url=URL)
    soup = bs(data.text, 'xml')
    names = soup.find_all("name")
    
    for name in names:
        if (SequenceMatcher(a=response,b=name.text).ratio()):
            #print(name.text)
            #print(name.parent.find("appID").text)
            return name.parent.find("appID").text
    
    print("Game not found")

    #print("hello")



    


In [81]:
    
# From here
def qCommand(x):

    x = x.lower()
    r2 = sr.Recognizer()
   
    if 'youtube' in x:
    
           
            driver.get('https://www.youtube.com/')

            with sr.Microphone() as source:
                engine.say("What do you want to see?")
                engine.runAndWait()
                print('')
                audio = r2.listen(source)
                keyword = r2.recognize_google(audio)
                print(keyword)
      

            YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'
 
            response = callGPT(YOUR_PROMPT)



            elem = driver.find_element('xpath', '//input[@id="search"]')
            elem.click()
            elem.send_keys(response, Keys.RETURN)

                

            try:
                get = r2.recognize_google(audio)
                print(get)
            except sr.UnknownValueError:
                print('Client side error')
            except sr.RequestError:
                print('Error on my side')

            # Till here is the code to run a YouTube vid
    elif 'google' in x:
           
            driver.get('https://www.google.com/')

            with sr.Microphone() as source:
                engine.say("What do you want to search for?")
                engine.runAndWait()
                print('')
                audio = r2.listen(source,timeout=5)
                keyword = r2.recognize_google(audio)
                print(keyword)
            YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'

            completion = openai.Completion.create(
                model=cur_model,
                prompt=YOUR_PROMPT,
                temperature=0,
                frequency_penalty=0.5,
                max_tokens=20,


            )

            response = callGPT(YOUR_PROMPT)



            elem = driver.find_element('xpath', '//input[@class="gLFyf gsfi"]')
            elem.click()
            elem.send_keys(response, Keys.RETURN)
            
            # if img in x:

                
    elif 'play' in x:

        print(x)
        
        prompt = "Extract the name of the game from the prompt: " + x

        response = callGPT(prompt)

        gameID = getGame(response)

        driver.get("steam://rungameid/" + gameID)

    else:
            print("command not recognized")


    



In [82]:

r1 = sr.Recognizer()
r2 = sr.Recognizer()
r3 = sr.Recognizer()

engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)

with sr.Microphone() as source:
    print('Listening...')
    engine.say("Hello. I am Idaw. What can I do for you today?")
    engine.runAndWait()
    audio = r3.listen(source)

qCommand(r2.recognize_google(audio))


Listening...

I want to see a video about best Jake combos
I want to see a video about best Jake combos


# Voice Assist